<a href="https://colab.research.google.com/github/apple-pie-h/Health-Risk-Classifier-using-ML/blob/main/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =====================================================
# 1. IMPORTS
# =====================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss

In [2]:
# =====================================================
# 2. LOAD DATA
# =====================================================
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")

test_ids = test_df["record_id"]

In [3]:
# =====================================================
# 3. GLOBAL BOOLEAN CLEANUP (CRITICAL)
# =====================================================
bool_map = {
    "TRUE": 1, "FALSE": 0,
    "True": 1, "False": 0,
    "YES": 1, "NO": 0,
    "Yes": 1, "No": 0,
    True: 1, False: 0
}

for df in [train_df, test_df]:
    df.replace(bool_map, inplace=True)

In [4]:
# =====================================================
# 4. FIX WEEK COLUMN (W1, W12, etc.)
# =====================================================
def clean_week(col):
    return (
        col.astype(str)
           .str.upper()
           .str.replace("W", "", regex=False)
           .str.strip()
    )

for df in [train_df, test_df]:
    if "week" in df.columns:
        df["week"] = clean_week(df["week"])
        df["week"] = pd.to_numeric(df["week"], errors="coerce")

In [5]:
# =====================================================
# 5. NUMERIC COLUMN COERCION
# =====================================================
numeric_cols = [
    "temperature_celsius", "pm25_ugm3", "precipitation_mm",
    "air_quality_index", "heat_wave_days", "extreme_weather_events",
    "population_millions", "latitude", "longitude",
    "gdp_per_capita_usd", "mental_health_index",
    "food_security_index", "healthcare_access_index",
    "temp_anomaly_celsius", "year", "month", "week"
]

for col in numeric_cols:
    if col in train_df.columns:
        train_df[col] = pd.to_numeric(train_df[col], errors="coerce")
        test_df[col]  = pd.to_numeric(test_df[col], errors="coerce")

In [6]:
# =====================================================
# 6. DROP IDS / TEXT
# =====================================================
DROP_COLS = ["record_id", "country_name", "date"]

train_df.drop(columns=DROP_COLS, inplace=True, errors="ignore")
test_df.drop(columns=DROP_COLS, inplace=True, errors="ignore")

In [7]:
# =====================================================
# 7. ONE-HOT ENCODE CATEGORICALS
# =====================================================
cat_cols = ["country_code", "region", "income_level"]

train_df = pd.get_dummies(train_df, columns=cat_cols, drop_first=True)
test_df  = pd.get_dummies(test_df,  columns=cat_cols, drop_first=True)

train_df, test_df = train_df.align(test_df, join="left", axis=1, fill_value=0)

In [8]:
# =====================================================
# 8. ADD BIN FEATURES (RF BOOST)
# =====================================================
for df in [train_df, test_df]:
    if "pm25_ugm3" in df.columns:
        df["pm25_bin"] = pd.cut(
            df["pm25_ugm3"],
            bins=[0, 12, 35, 55, 150, 500],
            labels=False
        )

    if "air_quality_index" in df.columns:
        df["aqi_bin"] = pd.cut(
            df["air_quality_index"],
            bins=[0, 50, 100, 150, 200, 500],
            labels=False
        )

In [9]:
# =====================================================
# 9. SPLIT TARGET
# =====================================================
y = train_df["health_risk_class"]
X = train_df.drop("health_risk_class", axis=1)

# Ensure test has no target
test_df = test_df.drop(columns=["health_risk_class"], errors="ignore")

In [10]:
# =====================================================
# 10. HARD NUMERIC ENFORCEMENT
# =====================================================
X = X.apply(pd.to_numeric, errors="coerce")
test_df = test_df.apply(pd.to_numeric, errors="coerce")

In [11]:
# =====================================================
# 11. IMPUTATION
# =====================================================
imputer = SimpleImputer(strategy="median")

X[:] = imputer.fit_transform(X)
test_df[:] = imputer.transform(test_df)

assert X.select_dtypes(include="object").empty
assert not np.isnan(X.values).any()

/tmp/ipython-input-2646824252.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  X[:] = imputer.fit_transform(X)
/tmp/ipython-input-2646824252.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  X[:] = imputer.fit_transform(X)
/tmp/ipython-input-2646824252.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  X[:] = imputer.fit_transform(X)
/tmp/ipython-input-2646824252.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated 

In [12]:
# =====================================================
# 12. TRAIN / VALIDATION SPLIT
# =====================================================
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [13]:
# =====================================================
# 13. RANDOM FOREST (TUNED FOR DATA)
# =====================================================
rf = RandomForestClassifier(
    n_estimators=1400,
    max_depth=20,
    min_samples_leaf=6,
    min_samples_split=15,
    max_features=0.7,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20, max_features=0.7,
                       min_samples_leaf=6, min_samples_split=15,
                       n_estimators=1400, n_jobs=-1, random_state=42)

In [14]:
# =====================================================
# 14. VALIDATION METRICS
# =====================================================
val_pred  = rf.predict(X_val)
val_probs = rf.predict_proba(X_val)

print("Validation Accuracy:", accuracy_score(y_val, val_pred))
print("Validation Log Loss:", log_loss(y_val, val_probs))

Validation Accuracy: 0.5901128843908136
Validation Log Loss: 0.9137349779683369


In [15]:
# =====================================================
# 15. TRAIN FULL MODEL
# =====================================================
rf.fit(X, y)

RandomForestClassifier(class_weight='balanced', max_depth=20, max_features=0.7,
                       min_samples_leaf=6, min_samples_split=15,
                       n_estimators=1400, n_jobs=-1, random_state=42)

In [16]:
# =====================================================
# 16. TEST PREDICTIONS
# =====================================================
test_probs = rf.predict_proba(test_df)

In [17]:
# =====================================================
# 17. SUBMISSION
# =====================================================
submission = pd.DataFrame({
    "record_id": test_ids,
    "Low_Risk": test_probs[:, 0],
    "Moderate_Risk": test_probs[:, 1],
    "High_Risk": test_probs[:, 2],
    "Critical_Risk": test_probs[:, 3],
})

submission.to_csv("submission.csv", index=False)

print("✅ submission.csv created successfully")

✅ submission.csv created successfully
